### Importing packages

In [ ]:
import ansys.fluent.core as pyfluent
meshing_session = pyfluent.launch_fluent(precision="double", processor_count=2, mode="meshing", show_gui=True)
workflow = meshing_session.workflow
meshing = meshing_session.meshing

### Initializing workflow for meshing

In [ ]:
workflow.InitializeWorkflow(WorkflowType=r'Watertight Geometry')
meshing.GlobalSettings.LengthUnit.set_state(r'mm')
meshing.GlobalSettings.AreaUnit.set_state(r'mm^2')
meshing.GlobalSettings.VolumeUnit.set_state(r'mm^3')

### Import geometry

Replace file_location with the actual address of the Static Mixer geometry file.
###### Eg: workflow.TaskObject['Import Geometry'].Arguments.set_state({r'FileName': r'D:/PyFluent/Static Mixer geometry.dsco',})


In [ ]:
workflow.TaskObject['Import Geometry'].Arguments.set_state({r'FileName': r'file_location',})
workflow.TaskObject['Import Geometry'].Execute()

### Add local sizing

In [ ]:
workflow.TaskObject['Add Local Sizing'].AddChildAndUpdate()

### Generate the Surface Mesh

In [ ]:
workflow.TaskObject['Generate the Surface Mesh'].Execute()

### Describe geometry

In [ ]:
workflow.TaskObject['Describe Geometry'].UpdateChildTasks(SetupTypeChanged=False)
workflow.TaskObject['Describe Geometry'].Arguments.set_state({r'SetupType': r'The geometry consists of only fluid regions with no voids',})
workflow.TaskObject['Describe Geometry'].UpdateChildTasks(SetupTypeChanged=True)
workflow.TaskObject['Describe Geometry'].Execute()

### Updating boundaries

In [ ]:
workflow.TaskObject['Update Boundaries'].Execute()

### Updating regions

In [ ]:
workflow.TaskObject['Update Regions'].Execute()

### Add boundary layers

In [ ]:
workflow.TaskObject['Add Boundary Layers'].Arguments.set_state({r'LocalPrismPreferences': {r'Continuous': r'Stair Step',},})
workflow.TaskObject['Add Boundary Layers'].AddChildAndUpdate()

### Generate the volume Mesh

In [ ]:
workflow.TaskObject['Generate the Volume Mesh'].Execute()

### Switch to solution

In [ ]:
solver = meshing_session.switch_to_solver()

### Enabling energy

In [ ]:
solver.setup.models.energy = {"enabled" : True}

### Copying Water_liquid to materials panel

In [ ]:
solver.execute_tui(r'''/define/materials/copy fluid water-liquid ''')

### Assigning water_liquid to fluid domain

In [ ]:
solver.setup.cell_zone_conditions.fluid['fluid'] = {"material" : "water-liquid"}

### Assigning boudary conditons

In [ ]:
solver.setup.boundary_conditions.velocity_inlet['velocity-inlet-1'].vmag = 1.
solver.setup.boundary_conditions.velocity_inlet['velocity-inlet-2'].vmag = 1.

### Hybrid Initialization

In [ ]:
solver.solution.initialization.hybrid_initialize()

### Setting 20 iterations

In [ ]:
solver.solution.run_calculation.iterate(iter_count = 20)

### Exiting solver

In [ ]:
solver.exit()